<a href="https://colab.research.google.com/github/sprieton/CIFAR10_LeNet5/blob/main/CIFAR10_LeNet5_JB_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project I: Understanding Calibration in CNNs

En este proyecto, investigaremos la calibración de las Redes Neuronales Convolucionales (CNNs) en un entorno de clasificación binaria.

**Objetivos:**
1.  Entrenar una CNN LeNet-5 desde cero para clasificar **pájaros** vs. **gatos** del dataset CIFAR-10.
2.  Evaluar la calibración del modelo utilizando diagramas de fiabilidad y el Expected Calibration Error (ECE).
3.  Implementar y evaluar el método de *Temperature Scaling* para mejorar la calibración del modelo.
4.  (Opcional) Repetir el experimento con un modelo pre-entrenado más grande.

## 1. Preparación del Entorno

Primero, importamos todas las librerías necesarias para el proyecto.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split

import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from tqdm.notebook import tqdm

# Configuración del dispositivo (GPU o CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

## 2. Carga y Preprocesamiento de Datos (CIFAR-10)

Cargaremos el dataset CIFAR-10, pero solo nos interesan dos clases: **gatos** (clase 3) y **pájaros** (clase 2). Filtraremos el dataset para quedarnos únicamente con estas dos clases y re-etiquetaremos las clases a 0 (pájaro) y 1 (gato) para nuestro problema de clasificación binaria.

In [ ]:
# Transformaciones para las imágenes de CIFAR-10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Descargar y cargar el dataset de entrenamiento
trainset_full = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Descargar y cargar el dataset de test
testset_full = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Clases que nos interesan: 2 (bird) y 3 (cat)
classes_to_keep = [2, 3]

def filter_dataset(dataset, classes):
    # Extraer índices de las clases que queremos mantener
    idx = [i for i, (img, label) in enumerate(dataset) if label in classes]
    # Crear un nuevo mapeo de etiquetas: bird (2) -> 0, cat (3) -> 1
    dataset.targets = [0 if dataset.targets[i] == classes[0] else 1 for i in idx]
    # Crear el subconjunto del dataset
    return Subset(dataset, idx)

# Filtrar los datasets
trainset_filtered = filter_dataset(trainset_full, classes_to_keep)
testset = filter_dataset(testset_full, classes_to_keep)

# Dividir el conjunto de entrenamiento filtrado en entrenamiento y validación (80/20)
# Usaremos el set de validación para encontrar el parámetro 'a' de Temperature Scaling
train_size = int(0.8 * len(trainset_filtered))
val_size = len(trainset_filtered) - train_size
trainset, valset = random_split(trainset_filtered, [train_size, val_size])

# Crear los DataLoaders
batch_size = 64
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f'Tamaño del set de entrenamiento: {len(trainset)} imágenes')
print(f'Tamaño del set de validación: {len(valset)} imágenes')
print(f'Tamaño del set de test: {len(testset)} imágenes')

class_names = ['pájaro', 'gato']

### Visualización de Datos
Mostramos algunas imágenes para verificar que el filtrado y la carga se han realizado correctamente.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # Desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtener un batch de imágenes de entrenamiento
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Mostrar imágenes
imshow(torchvision.utils.make_grid(images[:8]))
# Imprimir etiquetas
print(' '.join(f'{class_names[labels[j]]:5s}' for j in range(8)))

## 3. Definición del Modelo (LeNet-5)

Definimos la arquitectura LeNet-5, adaptándola para las imágenes de CIFAR-10 (32x32x3) y para una clasificación binaria. La última capa tendrá una sola salida, que representará el logit para la clase 'gato'.

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # El input es 32x32x3
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5) # -> 28x28x6
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # -> 14x14x6
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5) # -> 10x10x16
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # -> 5x5x16

        # Las capas fully connected
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        # Capa de salida para clasificación binaria (1 logit)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = LeNet5().to(device)

## 4. Entrenamiento del Modelo

Definimos la función de pérdida y el optimizador. Usaremos `BCEWithLogitsLoss`, que es adecuada para clasificación binaria y numéricamente estable, ya que combina una sigmoide con la Binary Cross Entropy.

Luego, creamos el bucle de entrenamiento.

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f'Epoch {epoch+1}/{num_epochs}'), 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}')

print('Finished Training')

## 5. Evaluación de la Calibración del Modelo Original

Ahora que el modelo está entrenado, evaluaremos su rendimiento y, lo más importante, su calibración. Para ello, necesitamos obtener las probabilidades predichas y las etiquetas reales del conjunto de test.

In [ ]:
def get_predictions(model, dataloader):
    model.eval()
    all_labels = []
    all_probs = []
    all_logits = []

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            logits = model(images)
            probs = torch.sigmoid(logits) # Convertir logits a probabilidades

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu.numpy().flatten())
            all_logits.extend(logits.cpu().numpy().flatten())

    return np.array(all_labels), np.array(all_probs), np.array(all_logits)

In [ ]:
# Obtener predicciones del conjunto de test
true_labels, probs_uncalibrated, logits_uncalibrated = get_predictions(net, testloader)

# Calcular accuracy
preds = (probs_uncalibrated > 0.5).astype(int)
accuracy = np.mean(preds == true_labels)
print(f'Accuracy en el test set: {accuracy * 100:.2f}%')

### Diagrama de Fiabilidad y ECE

El diagrama de fiabilidad (reliability diagram) nos permite visualizar qué tan bien calibrado está un modelo. Compara la confianza media de las predicciones (eje x) con la precisión real de esas predicciones (eje y). En un modelo perfectamente calibrado, la gráfica sería la línea diagonal y=x.

El **Expected Calibration Error (ECE)** cuantifica esta desviación. Es la diferencia media ponderada entre la precisión y la confianza en varios "bins" o intervalos de confianza.

In [ ]:
def plot_reliability_diagram(true_labels, probs, n_bins=10):
    """Plots a reliability diagram and calculates ECE."""
    prob_true, prob_pred = calibration_curve(true_labels, probs, n_bins=n_bins, strategy='uniform')

    # Calcular ECE
    bin_counts, _ = np.histogram(probs, n_bins, range=(0, 1))
    bin_weights = bin_counts / len(probs)
    ece = np.sum(bin_weights * np.abs(prob_true - prob_pred))

    # Plot
    plt.figure(figsize=(8, 8))
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly calibrated')

    # Dibujar las barras de confianza vs. precisión
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.bar(bin_centers, prob_true, width=1.0/n_bins, edgecolor='black', alpha=0.6, label='Model output')

    plt.xlabel('Confidence (Predicted Probability)')
    plt.ylabel('Accuracy (True Probability in bin)')
    plt.title(f'Reliability Diagram (ECE = {ece:.4f})')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    return ece

print("Resultados del modelo sin calibrar:")
ece_uncalibrated = plot_reliability_diagram(true_labels, probs_uncalibrated)

## 6. Implementación de Temperature Scaling

Temperature Scaling es una técnica de calibración post-procesamiento muy simple. Consiste en escalar los logits del modelo por un parámetro `a` (o dividirlos por una "temperatura" `T`, donde `a = 1/T`) antes de aplicar la función sigmoide.

$$ \text{logit}_{\text{scaled}} = a \cdot \text{logit}_{\text{original}} $$

El parámetro `a` se optimiza para minimizar la Negative Log-Likelihood (NLL) o, en nuestro caso, la `BCEWithLogitsLoss`, en un **conjunto de validación**.

In [ ]:
# Primero, obtenemos los logits y etiquetas del conjunto de validación
val_labels, _, val_logits = get_predictions(net, valloader)

# Convertimos a tensores de PyTorch para la optimización
val_logits_tensor = torch.tensor(val_logits, dtype=torch.float32, device=device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.float32, device=device).view(-1, 1)

# Definimos 'a' como un parámetro entrenable
a = nn.Parameter(torch.ones(1, device=device))

# Usamos un optimizador para encontrar el mejor 'a'
optimizer_a = optim.LBFGS([a], lr=0.01, max_iter=50)

def eval_a():
    optimizer_a.zero_grad()
    loss = criterion(val_logits_tensor * a, val_labels_tensor)
    loss.backward()
    return loss

optimizer_a.step(eval_a)

optimal_a = a.item()
print(f'El valor óptimo para \'a\' es: {optimal_a:.4f}')

### Evaluación de la Calibración después de Temperature Scaling

Con el valor óptimo de `a`, escalamos los logits del conjunto de test y volvemos a calcular las probabilidades. Luego, generamos el nuevo diagrama de fiabilidad y el ECE.

In [ ]:
# Aplicar el escalado a los logits del conjunto de test
logits_calibrated = logits_uncalibrated * optimal_a

# Recalcular las probabilidades
probs_calibrated = 1 / (1 + np.exp(-logits_calibrated))

print("Resultados del modelo con Temperature Scaling:")
ece_calibrated = plot_reliability_diagram(true_labels, probs_calibrated)

## 7. Estudio del Efecto del Parámetro `a`

Vamos a visualizar cómo diferentes valores de `a` afectan a las probabilidades y al ECE. Un valor de `a < 1` (T > 1) "suaviza" las predicciones, haciéndolas menos confiadas. Un valor de `a > 1` (T < 1) las hace más "puntiagudas" o confiadas.

In [ ]:
a_values = [0.5, optimal_a, 1.5] # a < 1, a_optimo, a > 1
eces = []

for a_val in a_values:
    temp_probs = 1 / (1 + np.exp(-(logits_uncalibrated * a_val)))
    ece = plot_reliability_diagram(true_labels, temp_probs)
    print(f'Para a = {a_val:.4f}, ECE = {ece:.4f}')
    eces.append(ece)

## 8. (Opcional) Experimento con un Modelo Pre-entrenado

En esta sección, se repetiría el proceso utilizando un modelo más grande y complejo, como ResNet18, pre-entrenado en ImageNet.

**Pasos a seguir:**
1.  Cargar un modelo `resnet18` pre-entrenado desde `torchvision.models`.
2.  Congelar los pesos de todas las capas (`param.requires_grad = False`).
3.  Reemplazar la última capa (`fc`) por una nueva capa lineal con una sola salida para nuestra clasificación binaria. Esta será la única capa que se entrene.
4.  Ajustar las transformaciones de las imágenes (resize a 224x224 y normalización con las medias y desviaciones de ImageNet).
5.  Re-entrenar (fine-tuning) el modelo solo en la nueva capa.
6.  Repetir el análisis de calibración (diagrama de fiabilidad, ECE) antes y después de aplicar Temperature Scaling.

In [ ]:
# --- CELDA DE CÓDIGO PARA LA PARTE OPCIONAL ---
# Aquí iría la implementación de la parte opcional del proyecto.
# Puedes descomentar y completar las siguientes líneas como guía.

# 1. Cargar modelo pre-entrenado
# model_resnet = torchvision.models.resnet18(pretrained=True)

# 2. Congelar pesos
# for param in model_resnet.parameters():
#     param.requires_grad = False

# 3. Reemplazar la última capa
# num_ftrs = model_resnet.fc.in_features
# model_resnet.fc = nn.Linear(num_ftrs, 1)
# model_resnet = model_resnet.to(device)

# 4. Ajustar transformaciones (requeriría nuevos DataLoaders)
# ...

# 5. Entrenar y evaluar
# ...

print("Sección opcional. Implementar aquí.")